In [4]:
from gurobipy import *

In [2]:
import random
n = 10
m = 5
set_I = range(1, n+1)
set_J = range(1, m+1)
c = {(i,j): random.normalvariate(0,1) for i in set_I for j in set_J}
a = {(i,j): random.normalvariate(0,5) for i in set_I for j in set_J}
l = {(i,j): random.randint(0,10) for i in set_I for j in set_J}
u = {(i,j): random.randint(10,20) for i in set_I for j in set_J}
b = {j: random.randint(0,30) for j in set_J}

In [5]:
import gurobipy as grb
opt_model = grb.Model(name="MIP Model")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-04


In [6]:
# if x is Continuous
x_vars  ={(i,j):opt_model.addVar(vtype=grb.GRB.CONTINUOUS, 
                        lb=l[i,j], 
                        ub= u[i,j],
                        name="x_{0}_{1}".format(i,j)) 
for i in set_I for j in set_J}
# if x is Binary
x_vars  = {(i,j):opt_model.addVar(vtype=grb.GRB.BINARY,
                        name="x_{0}_{1}".format(i,j)) 
for i in set_I for j in set_J}
# if x is Integer
x_vars  ={(i,j):opt_model.addVar(vtype=grb.GRB.INTEGER,
                        lb=l[i,j], 
                        ub= u[i,j],
                        name="x_{0}_{1}".format(i,j)) 
for i in set_I for j in set_J}

In [7]:
# <= constraints
constraints = {j : 
opt_model.addConstr(
        lhs=grb.quicksum(a[i,j] * x_vars[i,j] for i in set_I),
        sense=grb.GRB.LESS_EQUAL,
        rhs=b[j], 
        name="constraint_{0}".format(j))
    for j in set_J}
# >= constraints
constraints = {j : 
opt_model.addConstr(
        lhs=grb.quicksum(a[i,j] *x_vars[i,j] for i in set_I),
        sense=grb.GRB.GREATER_EQUAL,
        rhs=b[j], 
        name="constraint_{0}".format(j))
    for j in set_J}
# == constraints
constraints = {j : 
opt_model.addConstr(
        lhs=grb.quicksum(a[i,j] * x_vars[i,j] for i in set_I),
        sense=grb.GRB.EQUAL,
        rhs=b[j], 
        name="constraint_{0}".format(j))
    for j in set_J}

<ipython-input-7-6b16afa5f889>:2: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  constraints = {j :
<ipython-input-7-6b16afa5f889>:10: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  constraints = {j :
<ipython-input-7-6b16afa5f889>:18: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  constraints = {j :


In [8]:
objective = grb.quicksum(x_vars[i,j] * c[i,j] 
                         for i in set_I 
                         for j in set_J)

In [9]:
# for maximization
opt_model.ModelSense = grb.GRB.MAXIMIZE
# for minimization
opt_model.ModelSense = grb.GRB.MINIMIZE
opt_model.setObjective(objective)

In [10]:
opt_model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15 rows, 150 columns and 150 nonzeros
Model fingerprint: 0x75a5037b
Variable types: 50 continuous, 100 integer (50 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+01]
  Objective range  [3e-03, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [6e+00, 3e+01]
Presolve removed 1 rows and 101 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [12]:
import pandas as pd
opt_df = pd.DataFrame.from_dict(x_vars, orient="index", 
                                columns = ["variable_object"])
opt_df.index = pd.MultiIndex.from_tuples(opt_df.index, 
                               names=["column_i", "column_j"])
opt_df.reset_index(inplace=True)

In [13]:
# Gurobi
opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.X)

AttributeError: Unable to retrieve attribute 'X'

In [1]:
# Solve the following MIP:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp

# Create a new model
m = gp.Model()

# Create variables
x = m.addVar(vtype='B', name="x")
y = m.addVar(vtype='B', name="y")
z = m.addVar(vtype='B', name="z")

# Set objective function
m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# Add constraints
m.addConstr(x + 2 * y + 3 * z <= 4)
m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-04
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0
